In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import simpy as sp
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
from sympy import symbols, Matrix
m,rx,ry,rz,vx,vy,vz,q1,q2,q3,q4,w1,w2,w3 = symbols('m,rx,ry,rz,vx,vy,vz,q1,q2,q3,q4,w1,w2,w3')
alpha_m, r_t = symbols('alpha_m,r_t')

In [ ]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
import Landing3DModel
import Landing3DCost
import Landing3DConstraints
from scipy.integrate import solve_ivp
from Scvx import Scvx

In [ ]:
ix = 14
iu = 3
ih = 7
tf = 5
N = 50
delT = tf/N
max_iter = 15

In [ ]:
xi = np.array([2,1,1,2,0,0,0,1,0,0,0,0,0,0])
xf = np.array([1,0,0,0,0,0,0,1,0,0,0,0,0,0])

myModel = Landing3DModel.Landing3D('Hello',ix,iu,delT)
myCost = Landing3DCost.Landing3D('Hello',ix,iu,N)
myConst = Landing3DConstraints.Landing3D('Hello',ix,iu,ih)

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,2] = 2

In [ ]:
idx = 2
fx,fu = myModel.diff_numeric(x0[0],u0[0])
print(fu[idx])
fx,fu = myModel.diff_numeric_central(x0[0],u0[0])
print(fu[idx])